<a href="https://colab.research.google.com/github/aviichal/CryptoVisionAI/blob/main/CryptoVisionAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing Necessary Libraries
import hashlib
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

# Simulating In-Memory Databases
users_db = {}
portfolio_db = {}

#  Defining Supported Cryptocurrencies
cryptos = {
    '1': {'name': 'Bitcoin', 'id': 'bitcoin'},
    '2': {'name': 'Ethereum', 'id': 'ethereum'},
    '3': {'name': 'Litecoin', 'id': 'litecoin'},
    '4': {'name': 'Ripple', 'id': 'ripple'},
    '5': {'name': 'Cardano', 'id': 'cardano'}
}

#User Registration and Authentication

# Password hashing for security
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Registering a New User
def register_user():
    email = input("Enter your email: ")
    if email in users_db:
        print("User already exists.")
        return
    password = input("Enter your password: ")
    full_name = input("Enter your full name: ")
    bank_balance = float(input("Enter your initial bank balance: "))
    users_db[email] = {
        'password': hash_password(password),
        'full_name': full_name,
        'bank_balance': bank_balance,
        'crypto_held': []
    }
    portfolio_db[email] = []
    print("Registration successful.\n")

# Logging in an Existing User
def login_user():
    email = input("Enter your email: ")
    password = input("Enter your password: ")
    if email in users_db and users_db[email]['password'] == hash_password(password):
        print(f"Welcome, {users_db[email]['full_name']}!\n")
        return email
    else:
        print("Invalid email or password.\n")
        return None

#Portfolio Management

# Adding Assets to the Portfolio
def add_to_portfolio(email, crypto_name, quantity, purchase_price):
    portfolio_db[email].append({
        'crypto_name': crypto_name,
        'quantity': quantity,
        'purchase_price': purchase_price
    })

# Displaying the User's Portfolio
def display_portfolio(email):
    portfolio = portfolio_db.get(email, [])
    if not portfolio:
        print("No assets in portfolio.")
        return
    for asset in portfolio:
        print(f"{asset['crypto_name']}: {asset['quantity']} units at ${asset['purchase_price']} each\n")

# Manual trading
def manual_trade(email):
    print("Choose a cryptocurrency:")
    for key, crypto in cryptos.items():
        print(f"{key}. {crypto['name']}")
    crypto_choice = input("Enter the number for the crypto: ")
    if crypto_choice not in cryptos:
        print("Invalid choice.\n")
        return

    crypto_name = cryptos[crypto_choice]['name']
    quantity = float(input(f"Enter quantity of {crypto_name}: "))
    price = float(input(f"Enter price of {crypto_name}: "))
    user = users_db.get(email)

    if not user:
        print("User not found.")
        return

    cost = quantity * price
    if user['bank_balance'] < cost:
        print("Insufficient funds.\n")
        return

    user['bank_balance'] -= cost
    add_to_portfolio(email, crypto_name, quantity, price)
    print(f"Bought {quantity} units of {crypto_name} at ${price}.\n")

# Fetch historical crypto data from CoinGecko API
def fetch_crypto_data(crypto_id='bitcoin', days='365'):
    url = f'https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart?vs_currency=usd&days={days}'
    response = requests.get(url).json()
    prices = response['prices']
    df = pd.DataFrame(prices, columns=['timestamp', 'price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# LSTM-based forecasting model
def lstm_forecast(data, days_to_predict=30):
    data = data.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    # Prepare the data for LSTM
    X_train = []
    y_train = []
    for i in range(60, len(data_scaled)):
        X_train.append(data_scaled[i-60:i, 0])
        y_train.append(data_scaled[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=1)

    # Predict future prices
    test_data = data_scaled[-60:]  # Last 60 days
    X_test = []
    X_test.append(test_data)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    predictions = []
    for _ in range(days_to_predict):
        predicted_price = model.predict(X_test)
        predictions.append(predicted_price[0][0])
        # Shift the window by adding the predicted value
        X_test = np.roll(X_test, -1, axis=1)
        X_test[0, -1, 0] = predicted_price[0][0]

    # Inverse scale the predictions
    predicted_prices = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    return predicted_prices

# Plotting predicted prices
def plot_predictions(df, predicted_prices, days_to_predict, crypto_name):
    future_dates = pd.date_range(df['timestamp'].iloc[-1], periods=days_to_predict + 1, freq='D')[1:]
    for date, price in zip(future_dates, predicted_prices):
        print(f"Predicted price for {crypto_name} on {date.date()}: ${price[0]:.2f}")

# Auto-buy logic based on user-specified profit threshold OR Automated Buying Strategy (Crypto Assistant)
def auto_buy_logic(email):
    print("Choose a cryptocurrency:")
    for key, crypto in cryptos.items():
        print(f"{key}. {crypto['name']}")
    crypto_choice = input("Enter the number for the crypto: ")
    if crypto_choice not in cryptos:
        print("Invalid choice.\n")
        return

    crypto_name = cryptos[crypto_choice]['name']
    crypto_id = cryptos[crypto_choice]['id']

    df = fetch_crypto_data(crypto_id, '365')
    current_price = df['price'].iloc[-1]

    predict_days = int(input("Enter the number of days for prediction: "))
    lstm_predictions = lstm_forecast(df['price'], predict_days)

    # Display predicted prices
    plot_predictions(df, lstm_predictions, predict_days, crypto_name)

    future_price = lstm_predictions[-1]  # Last predicted price
    user_profit_target = float(input("Enter the desired profit percentage: "))

    user = users_db.get(email)
    if not user:
        print("User not found.\n")
        return

    expected_profit = ((future_price - current_price) / current_price) * 100
    if isinstance(expected_profit, np.ndarray):
        expected_profit = expected_profit.item()  # Convert array element to a scalar
    expected_profit = float(expected_profit)  # Ensure it's a float

    if expected_profit > user_profit_target:
        amount_to_invest = user['bank_balance'] * 0.5
        quantity = amount_to_invest / current_price
        user['bank_balance'] -= amount_to_invest
        add_to_portfolio(email, crypto_name, quantity, current_price)
        print(f"\nAuto-bought {quantity} units of {crypto_name} at ${current_price:.2f}.\n")
    else:
        print(f"\nExpected profit of {expected_profit:.2f}% does not meet the desired {user_profit_target}% threshold.\n")

# Fraud detection using Isolation Forest
def detect_fraud(email):
    portfolio = portfolio_db.get(email, [])
    if len(portfolio) < 2:
        print("Not enough trades to detect fraud.\n")
        return

    X = np.array([[trade['quantity'], trade['purchase_price']] for trade in portfolio])
    model = IsolationForest(contamination=0.1)
    model.fit(X)
    anomalies = model.predict(X)

    for i, is_fraud in enumerate(anomalies):
        if is_fraud == -1:
            print(f"Fraud detected in trade: {portfolio[i]}")

# User Interface:Main menu-driven interface
def menu():
    logged_in_email = None

    while True:
        if logged_in_email is None:
            print("\n--- Main Menu ---")
            print("1. Register")
            print("2. Login")
            print("3. Exit")
            choice = input("Choose an option: ")

            if choice == '1':
                register_user()
            elif choice == '2':
                logged_in_email = login_user()
            elif choice == '3':
                print("Exiting...")
                break
            else:
                print("Invalid choice. Please try again.")
        else:
            print("\n--- Dashboard ---")
            print("1. Manual Trade")
            print("2. View Portfolio")
            print("3. Run Crypto Assistant (Auto-buy)")
            print("4. Detect Fraud")
            print("5. Logout")
            choice = input("Choose an option: ")

            if choice == '1':
                manual_trade(logged_in_email)
            elif choice == '2':
                display_portfolio(logged_in_email)
            elif choice == '3':
                auto_buy_logic(logged_in_email)
            elif choice == '4':
                detect_fraud(logged_in_email)
            elif choice == '5':
                logged_in_email = None
                print("Logged out successfully.")
            else:
                print("Invalid choice. Please try again.")

#Program Entry Point

if __name__ == "__main__":
    menu()



--- Main Menu ---
1. Register
2. Login
3. Exit
Registration successful.


--- Main Menu ---
1. Register
2. Login
3. Exit
Welcome, safaan shawl!


--- Dashboard ---
1. Manual Trade
2. View Portfolio
3. Run Crypto Assistant (Auto-buy)
4. Detect Fraud
5. Logout
Choose a cryptocurrency:
1. Bitcoin
2. Ethereum
3. Litecoin
4. Ripple
5. Cardano


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


306/306 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - loss: 0.0198
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

<ipython-input-10-a1e745e86b6c>:196: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  expected_profit = float(expected_profit)  # Ensure it's a float



Auto-bought 20390.575568138975 units of Ethereum at $2452.11.


--- Dashboard ---
1. Manual Trade
2. View Portfolio
3. Run Crypto Assistant (Auto-buy)
4. Detect Fraud
5. Logout


KeyboardInterrupt: Interrupted by user